Best place to dine at in New Delhi

Table of contents
1)Introduction
2)Data acquisition
3)Methodology
4)Analysis
5)Results and Discussion
6)Conclusion


1. Introduction 
1.
Delhi is a big city and new restaurants open up everyday.The intent is to analyze Types of restaurants & their locations 
1.2 Problem
For some one who wants to dine at the best restaurants in Delhi, this is a one stop solution.


In [20]:
# Import statements
import numpy as np 
import pandas as pd 
import json

#!conda install -c conda-forge geopy --yes 
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!pip install seaborn
import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
#import folium # map rendering library

print("Libraries Imported.")

Libraries Imported.


2. Data acquisition 
2.1 Source of Data

a) Foursquare API to select the number of restaurants and their locations in New Delhi.
b) Zomato API to fetch ratings and average price of the restaurants .

I will first create a dataset thought the Foursquare API to explore several types of venues, such as ID, name, latitude, longitude, neighborhood, and distance (in meters) to Connaught place which is a hub for restaurants in Delhi.

#### Define Foursquare Credentials and Version


In [21]:
# @hidden_cell
CLIENT_ID = '5ELJDSHQBU32TMQFTJAVJQV44MEHWQ0MYNXMJWDGGBKCUGG2' # your Foursquare ID
CLIENT_SECRET = '0SMNP3OLJ2CGE0MRBCWSCWHD45CESHNXYTRWOWDSH2YB1GSF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [ ]:
LIMIT = 15000 # A default Foursquare API limit value
radius = 1000

In [22]:
# Coorinates for Connaught Place abbreviated as CP
CP_latitude = 28.6304
CP_longitude = 77.2177


Using the Foursquare API

In [61]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    CP_latitude, 
    CP_longitude, 
    radius, 
    LIMIT,
)

In [62]:
results = requests.get(url).json()
results.values();
results

{'meta': {'code': 200, 'requestId': '5fb35e6b0cc1d6096feb27f7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Delhi',
  'headerFullLocation': 'Delhi',
  'headerLocationGranularity': 'city',
  'totalResults': 193,
  'suggestedBounds': {'ne': {'lat': 28.72040009000009,
    'lng': 77.32004601237499},
   'sw': {'lat': 28.540399909999913, 'lng': 77.115353987625}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc8f2c7762beee1a8bb3d38',
       'name': 'The Imperial',
       'location': {'address': 'Janpath',
        'lat': 28.625548038159426,
        'lng': 77.21866351780929,
        'labeledLatLngs': [{'label': 'display',
          'lat': 28.625548038159426,
          'lng': 77.218

In [63]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [67]:
venues = results['response']['groups'][0]['items']
venues_neighborhood = [results['response']['groups'][0]['items'][n]['venue']['location']['formattedAddress'] for n in range(len(venues))]  
venues_distance = [results['response']['groups'][0]['items'][n]['venue']['location']['distance'] for n in range(len(venues))]  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

[venues_neighborhood[k][0] for k in range(len(venues_neighborhood))]

#nearby_venues['neighborhood'] = venues_neighborhood[0][0]
nearby_venues['neighborhood'] =  [venues_neighborhood[k][0] for k in range(len(venues_neighborhood))]
nearby_venues['distance [m]'] = venues_distance

df = nearby_venues
df

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,lat,lng,neighborhood,distance [m]
0,4bc8f2c7762beee1a8bb3d38,The Imperial,Hotel,28.625548,77.218664,Janpath,548
1,4b489b54f964a520595026e3,Connaught Place | कनॉट प्लेस (Connaught Place),Plaza,28.632731,77.220018,Connaught Place (Many streets meet here),344
2,54783eab498e910f8bd1781d,Naturals Ice Cream,Ice Cream Shop,28.634455,77.222139,L Block Outer Circle (Connaught Place),626
3,519ba450498eb0c559152d94,HOTEL SARAVANA BHAVAN,South Indian Restaurant,28.632319,77.216445,"P-13/90, Connaught Circus",246
4,54dc85c7498ef8f9ab9b3c08,Tamra,Restaurant,28.620543,77.218174,"19, Ashoka Rd (Windsor Pl)",1098
...,...,...,...,...,...,...,...
95,530af53a498e95adf56d82ce,Diggin,Café,28.555665,77.218590,1&2 Anand Lok shopping center (Opp Gargi college),8319
96,4f4e4436e4b088c4af2f575f,Concept Shawarma,Fast Food Restaurant,28.578280,77.238896,India,6160
97,52cc447f11d230fd22170cd1,Hyatt Regency Delhi,Hotel,28.569264,77.185290,Bhikaiji Cama Place Ring Road,7506
98,4dff63787d8bc001316c3a96,Hauz Khas Village,Market,28.554311,77.195140,Haus Khas (Near Greenpark Market),8752


In [65]:
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4bc8f2c7762beee1a8bb3d38',
   'name': 'The Imperial',
   'location': {'address': 'Janpath',
    'lat': 28.625548038159426,
    'lng': 77.21866351780929,
    'labeledLatLngs': [{'label': 'display',
      'lat': 28.625548038159426,
      'lng': 77.21866351780929}],
    'distance': 548,
    'postalCode': '110001',
    'cc': 'IN',
    'city': 'New Delhi',
    'state': 'Delhi',
    'country': 'India',
    'formattedAddress': ['Janpath', 'New Delhi 110001', 'Delhi', 'India']},
   'categories': [{'id': '4bf58dd8d48988d1fa931735',
     'name': 'Hotel',
     'pluralName': 'Hotels',
     'shortName': 'Hotel',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4bc8f2c7762beee1a8bb3d38-0'},
 {'r

In [ ]:
# @hidden_cell
headers = {'user-key': 'cd59127fe6022df6ce5e0e7f595a08c1'}

The Zomato API allows using its search API to search for any given venue based on certain search filters such as query, latitude, longitude and more. Zomato also requires a Zomato user key which can be accessed with a developer account.
We'll use the name, lat, and lng values of various venues fetched from Foursquare API to use the search API and get more information regarding each venue.
The query will be the name of the venue.


In [71]:

venues_information = []

for index, row in df.iterrows():
    venue = []
    url = ('https://developers.zomato.com/api/v2.1/search?q={}' + 
          '&start=0&count=1&lat={}&lon={}&sort=real_distance').format(row['name'], row['lat'], row['lng'])
    result = requests.get(url, headers = headers).json()
    if (len(result['restaurants']) > 0):
        venue.append(result['restaurants'][0]['restaurant']['name'])
        venue.append(result['restaurants'][0]['restaurant']['location']['latitude'])
        venue.append(result['restaurants'][0]['restaurant']['location']['longitude'])
        venue.append(result['restaurants'][0]['restaurant']['average_cost_for_two'])
        venue.append(result['restaurants'][0]['restaurant']['price_range'])
        venue.append(result['restaurants'][0]['restaurant']['user_rating']['aggregate_rating'])
        venue.append(result['restaurants'][0]['restaurant']['location']['address'])
        venues_information.append(venue)
    else:
        venues_information.append(np.zeros(6))
    
zomato_venues = pd.DataFrame(venues_information, 
                                  columns = ['venue', 'latitude', 
                                             'longitude', 'price_for_two', 
                                             'price_range', 'rating', 'address'])

In [74]:
zomato_venues

,venue,latitude,longitude,price_for_two,price_range,rating,address
0,Nostalgia at 1911 Brasserie - The Imperial,28.6250,77.2191,6000.0,4.0,3.6,"The Imperial, Janpath, New Delhi"
1,Kapurthala Kitchen,28.6327,77.2196,500.0,2.0,0,"N 35, Ground Floor, Chanakyapuri, Connaught Pl..."
2,Natural Ice Cream,28.6344,77.2220,350.0,1.0,4.7,"L-8, L Block, Outer Circle, Connaught Place, N..."
3,Saravana Bhavan,28.6323,77.2166,500.0,2.0,4.6,"P-13, Connaught Circus, Connaught Place, New D..."
4,Tamra - Shangri-La's - Eros Hotel,28.6211,77.2170,3800.0,4.0,4.2,"Shangri-La's - Eros Hotel, 19, Level 1, Ashoka..."
...,...,...,...,...,...,...,...
95,Diggin,28.5556,77.2184,1400.0,3.0,4.4,"Shop 1 & 2, Sri Fort Road, Opposite Gargi Coll..."
96,Concept Shawarma,28.5783,77.2388,650.0,2.0,4.2,"138/139, Flyover Market, Defence Colony, New D..."
97,T.K'S Oriental Grill - Hyatt Regency,28.5693,77.1859,4000.0,4.0,4.3,"Hyatt Regency, Bhikaji Cama Place, New Delhi"
98,Elma's Bakery & Kitchen,28.5544,77.1950,1500.0,3.0,4.0,"31, 2nd Floor, Hauz Khas Village, New Delhi"


In [75]:
df['lat'] = df['lat'].apply(lambda lat: round(float(lat), 4))
df['lng'] = df['lng'].apply(lambda lng: round(float(lng), 4))
zomato_venues['latitude'] = zomato_venues['latitude'].apply(lambda lat: round(float(lat), 4))
zomato_venues['longitude'] = zomato_venues['longitude'].apply(lambda lng: round(float(lng), 4))
dataset = pd.concat([df, zomato_venues], axis = 1)
dataset['lat_diff'] = dataset['latitude'] - dataset['lat']
dataset['lng_diff'] = dataset['longitude'] - dataset['lng']

In [76]:
selected_venues = dataset[(abs(dataset['lat_diff']) <= 0.0004) & (abs(dataset['lng_diff']) <= 0.0004)].reset_index(drop = True)
selected_venues

,id,name,categories,lat,lng,neighborhood,distance [m],venue,latitude,longitude,price_for_two,price_range,rating,address,lat_diff,lng_diff
0,4b489b54f964a520595026e3,Connaught Place | कनॉट प्लेस (Connaught Place),Plaza,28.6327,77.2200,Connaught Place (Many streets meet here),344,Kapurthala Kitchen,28.6327,77.2196,500.0,2.0,0,"N 35, Ground Floor, Chanakyapuri, Connaught Pl...",0.0000,-0.0004
1,54783eab498e910f8bd1781d,Naturals Ice Cream,Ice Cream Shop,28.6345,77.2221,L Block Outer Circle (Connaught Place),626,Natural Ice Cream,28.6344,77.2220,350.0,1.0,4.7,"L-8, L Block, Outer Circle, Connaught Place, N...",-0.0001,-0.0001
2,519ba450498eb0c559152d94,HOTEL SARAVANA BHAVAN,South Indian Restaurant,28.6323,77.2164,"P-13/90, Connaught Circus",246,Saravana Bhavan,28.6323,77.2166,500.0,2.0,4.6,"P-13, Connaught Circus, Connaught Place, New D...",0.0000,0.0002
3,4d655003823ca35d7f1cfe88,Jain Chawal Wale,Food Truck,28.6318,77.2158,Shivaji Stadium (Connaught Place),241,Jain Chawal Wale,28.6316,77.2161,250.0,1.0,4.1,"P-1/190, Connaught Circus, Connaught Place, Ne...",-0.0002,0.0003
4,4c91f42fae96a0932d79a746,HOTEL SARAVANA BHAVAN,South Indian Restaurant,28.6270,77.2195,"46, Janpath (Opp. Janpath Market)",413,Saravana Bhavan,28.6272,77.2197,500.0,2.0,4.4,"50, Atul Grove Road, Janpath, New Delhi",0.0002,0.0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,54df7735498edf03503c99d8,Diva Spiced,Asian Restaurant,28.5844,77.2267,"79-80 Meherchand Market, Lodhi Colony",5193,Carnatic Cafe,28.5843,77.2266,600.0,2.0,4.5,"84-85, Meharchand Market, Lodhi Road, Lodhi Co...",-0.0001,-0.0001
57,530af53a498e95adf56d82ce,Diggin,Café,28.5557,77.2186,1&2 Anand Lok shopping center (Opp Gargi college),8319,Diggin,28.5556,77.2184,1400.0,3.0,4.4,"Shop 1 & 2, Sri Fort Road, Opposite Gargi Coll...",-0.0001,-0.0002
58,4f4e4436e4b088c4af2f575f,Concept Shawarma,Fast Food Restaurant,28.5783,77.2389,India,6160,Concept Shawarma,28.5783,77.2388,650.0,2.0,4.2,"138/139, Flyover Market, Defence Colony, New D...",0.0000,-0.0001
59,4dff63787d8bc001316c3a96,Hauz Khas Village,Market,28.5543,77.1951,Haus Khas (Near Greenpark Market),8752,Elma's Bakery & Kitchen,28.5544,77.1950,1500.0,3.0,4.0,"31, 2nd Floor, Hauz Khas Village, New Delhi",0.0001,-0.0001


In [58]:
selected_venues['average_price'] = selected_venues['price_for_two']/2
selected_venues = selected_venues.drop(columns = ['name', 'lat', 'lng', 'lat_diff', 'lng_diff', 'price_for_two'])

In [70]:
selected_venues.head(10)

,id,categories,neighborhood,distance [m],venue,latitude,longitude,price_range,rating,address,average_price
0,4b489b54f964a520595026e3,Plaza,Connaught Place (Many streets meet here),344,Cafe Coffee Day,28.6330,77.2197,2.0,2.6,"Gate 7, Rajiv Chowk Metro Station, Connaught P...",250.0
1,54783eab498e910f8bd1781d,Ice Cream Shop,L Block Outer Circle (Connaught Place),626,Natural Ice Cream,28.6344,77.2220,1.0,4.7,"L-8, L Block, Outer Circle, Connaught Place, N...",175.0
2,519ba450498eb0c559152d94,South Indian Restaurant,"P-13/90, Connaught Circus",246,Saravana Bhavan,28.6323,77.2166,2.0,4.6,"P-13, Connaught Circus, Connaught Place, New D...",250.0
3,4d655003823ca35d7f1cfe88,Food Truck,Shivaji Stadium (Connaught Place),241,Jain Chawal Wale,28.6316,77.2161,1.0,4.1,"P-1/190, Connaught Circus, Connaught Place, Ne...",125.0
4,4c91f42fae96a0932d79a746,South Indian Restaurant,"46, Janpath (Opp. Janpath Market)",413,Saravana Bhavan,28.6272,77.2197,2.0,4.4,"50, Atul Grove Road, Janpath, New Delhi",250.0
5,56adb1f8498e9db1d6bb16ed,Café,"F block (Inner Circle, CP)",294,Chaayos - Meri Wali Chai,28.6316,77.2203,2.0,4.1,"F-14/15, 1st Floor, Mezzanine, F Block, Inner ...",250.0
6,4cb876d7f50e224bd2d6e6fb,Indian Restaurant,K Block (Connaught Place),635,Sagar Ratna,28.6354,77.2205,2.0,3.9,"K-15, Connaught Place, New Delhi",275.0
7,5113713fe4b09f5a3d6a7590,Coffee Shop,"A Block, Connaught Place (Hamilton House)",179,Starbucks Coffee,28.6322,77.2178,2.0,4.1,"A-1, Hamilton House, Inner Circle, Connaught P...",300.0
8,568feb91498edb833d755933,Molecular Gastronomy Restaurant,"Level 1, Block E, 38/39, Inner circle, Connaug...",413,Farzi Cafe,28.6325,77.2212,4.0,4.5,"38/39, Level 1, Block E, Inner Circle, Connaug...",1250.0
9,4bf107ab24f020a1e0a8684f,Indian Restaurant,"H 5 & 6, Plaza Building, Middle Circle, Connau...",525,Nizam's Kathi Kabab,28.6350,77.2195,2.0,4.1,"H-5/6, Plaza Building, Connaught Place, New Delhi",450.0
